In [7]:

from clifford.sta import * 
from pylab import * 

thres= 1e-12
is_close= lambda x,y:(x-y)<thres
split = lambda a,B: (a|B)/(a^B)
split_apart = lambda a,B: ((a|B)/B, (a^B)/B)
nudge = lambda a: a+1e-5 if a == 0 else a

# constants not defined by model
m = 3                   
q = 7
hbar = 4

# basis elements in standard frame
S = d12
i = d123 
I = d0123
k = i*S
C = I*S
pis = pi*S  # == \pi_S


## note: since we dont have a nice way to write the dot notation 
# we have set the differential (rdot) as rd ,  
#rd = d0 + .2*d1 + .84*d2 + .3*d3 # known differential
rd = D.randomV()            # random differential.
rd = rd(i) + abs(rd(i))*d0  # put it on the light cone


# differential components
rd0,rdk,rds = rd(d0),rd(k),rd(S)
rdsnot = rd-rds
rd0not = rd-rd0

# velocity
Vs,Vk = rds/rd0 ,rdk/rd0
V = Vs + Vk
T = Vk/Vs

# linear momentum 
ls,lk  = m*Vs,m*Vk  
l = ls+lk

rs = hbar/m*abs(Vs)* rds|S/abs(rds|S)  # position vector in S


# frequencies, angular momenta
theta = (rds/rs)
W = (rds/rs)/rd0
K = (rds/rdk)/rs
P = W + K
p,ps,pk = m*V,m*Vs, m*Vk
L = m*rs^Vs
E = m*V**2

# wavelengths
lam   = -2*pi*S*V/W
lam_s = -2*pi*S*Vs/W
lam_k = -2*pi*S*Vk/W

# currents
J  = q*V/lam
Js = q*Vs/lam_s
Jk = q*Vk/lam_k

# lorentz factor and helix space pitch 
Gamma = V*Vs.inv()         # spinor form 
gamma = abs(Gamma)   # lorentz factor
phi = arctan(abs(Vk)/abs(Vs))
Le = gamma**2 *L

# electrodynamics
M = Js*pis*rs**2
B = m/q*W
mu_ring = 4*pis*m*rs/q**2
Phi= B*pis*rs**2

if True:
    ################ TESTS ############################
    
    # Velocity and Differentials
    assert(rd == rd0+rdk+rds == rds+rdsnot)
    assert(-theta/(rd0*rdk)*(rd0+rdk) == P)      # P-r relation
    assert(  -rs*P == (rds*rdsnot)/(rd0*rdk))    # P-r relation
    assert(  Vs+Vk == rd0not/rd0)
    
    # Angular Momentum
    assert(L == ps^rs == -m*rs**2*W == -m*Vs**2/W)
    assert(isclose(abs(rs),abs(L/(m*Vs)))) # 
    assert(  rs**2 == - L/W/m)             # radius, as f(L,W,m)
    assert(T==Vk/Vs == rdk/rds== 1/(K*rs))
    assert(isclose(abs(T),tan(phi)))
    assert(Gamma==1+T)
     
    # Wavelengths
    assert(  lam == 2*pi*S*(rs+1/K))    # lamda = krds one  
    assert(lam_s == 2*pi*S*rs== 2*pis*L/ps) # deBroglie
    assert(lam_k == 2*pi*S*(1/K))
    assert(isclose(abs(lam)**2 ,abs(lam_s)**2 +abs(lam_k)**2))
    assert(lam==lam_s+lam_k)
    
    
    # Currents 
    assert(Js == -Jk ==-q*W/(2*pis))
    assert(isclose(abs(J),abs(Js)))
    assert(isclose(abs(J),abs(Jk)))
    assert(J==-q*W/(2*pis)*((1+T)/(1-T)))
    assert(J==-q*W/(2*pis)*V/(-k*V*k))
    assert(tan(phi)*(Vk/Vs)(2).normal() ==Vk/Vs)
    
    
    # Lorentz factor 
    assert(V**2 == Vs**2 +Vk**2)
    assert(Gamma==V*Vs.inv() == 1/(1-Vk*V.inv()))
    assert(is_close(gamma,1/sqrt(1-abs(Vk**2/V**2))))
    assert(is_close(gamma,1/cos(phi)))
       
    # Energy
    assert(E == m*V**2 == -L*W + m*Vk**2) # energy momentum
    assert(E ==-Le*W==-gamma**2*L*W)  
    assert(m*Vs**2 == -L*W )    
        
    # Magnetic moment 
    assert(M == -1/2*q*rs**2*W == q/(2*m)*L)
    assert(M==2*q/(2*m)*L/2)
    assert(L/M== 2*m/q)
    
    # Permeability
    assert(q*Vs*B== -rs*m*W**2)
    assert(B == mu_ring*q*W/(4*pis*rs))
    assert(mu_ring == 4*pis*m*rs/q**2 ==4/q**2*pis*L/Vs)
    
    #Flux  quantum 
    assert(L*W==2*B*M==2*Js*Phi)
    assert(Phi ==m/q*W*pis*rs**2==-pis*L/q)
    
    # Toroid
    assert(E==-L*W*(1-T**2) == -L*W*gamma**2)
    assert(-2*E/(Js**2*lam_k)==4*pis*m*rs/q**2*(T-1/T))
    assert(4*pis*m*rs/q**2*T.inv()==-2*L*W/(Js**2*lam_k))
    
    
    # Unfinished
    ## symmetries
    assert(-d0*P*d0 == -W + K) # T 
    assert(-i*P*i   == +W - K) # P?
    assert(-I*P*I   == -W - K) # C?
    assert(split(P*I,d0)== split(rdsnot,d0))

    ## null basis trivectords
    d0pkS = .5*(d0+d3)^S 
    d0mkS = .5*(d0-d3)^S
    Pp = (abs(W)+abs(K))*d0pkS  # probably a slicker way to do this
    Pm = (abs(W)-abs(K))*d0mkS
    #assert(P == Pm+Pp)


